# Проект
Данный проект нацелен на анализ вакансий от работодателей финансового сектора на сайте hh.ru
С целью сбора данных был написан специальный парсер (parser.ipynb)
В данном ноутбуке используется спарсенная заранее таблица с данными.

Предполагается сначала предварительно обработать датасет, затем уже описать содержащиеся внутри данные в процессе визуализации.

#### Импорт библиотек

In [426]:
import pandas as pd
import ast
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

#### Чтение датасета и предварительная обработка
Посмотрим как выглядят данные, удалим ненужные колонки, заполним NaN значения, преобразуем данные.
Начнём с чтения датасета. Так же посмотрим как он вообще выглядит.

In [427]:
df = pd.read_csv('fin-sec_15_05_2023.csv')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'fin-sec_15_05_2023.csv'

Давайте посмотрим что вообще находится внутри датасета.

In [ ]:
df.info()

Как можно заметить на данном этапе:
Колонки department, insider_interview содержат слишком мало данных для работы с ними.

Колонки response_url, sort_point_distance и adv_response_url, contacts, schedule пустые.

Среди данных, которые нам не нужны имеем следующее:
Unnamed: 0 - всё равно, что индекс.
Premium - не несёт практической пользы для анализа.
Name - необъективная информация.
Area - у нас и так только один регион.
Type - не несёт практической пользы для анализа.
Address - не несёт практической пользы для анализа.
Published_at - не несёт практической пользы для анализа.
Created_at - не несёт практической пользы для анализа.
Archived - не несёт практической пользы для анализа.
Apply_alternate_url - не несёт практической пользы для анализа.
Url - не несёт практической пользы для анализа.
Alternate_url - не несёт практической пользы для анализа.
Relations - не несёт практической пользы для анализа.
Employer - не несёт практической пользы для анализа.
Snippet - необъективная информация.
Working_days - неполные и/или нерелевантные данные.
Working_time_intervals - неполные и/или нерелевантные данные.
Working_time_modes - неполные и/или нерелевантные данные.

Соберём все выше указанные колонки в список и удалим.

In [ ]:
list_for_deletion = ['department', 'insider_interview', 'response_url', 'sort_point_distance', 'adv_response_url', 'contacts', 'schedule', 'Unnamed: 0', 'premium', 'name', 'area', 'type', 'address', 'published_at', 'created_at', 'archived', 'apply_alternate_url', 'url', 'alternate_url', 'relations', 'employer', 'snippet', 'working_days', 'working_time_intervals', 'working_time_modes']

df.drop(list_for_deletion, axis=1, inplace=True)
df.head()

В данном случае id вакансии уникальное значение, потому мы можем сделать её индексом.

In [ ]:
df.set_index('id', inplace=True, drop=True)
df.head()

Важно заметить, что колонки salary, professional_role, experience и employment представлены в виде словарей.
Эти данные необходимо вытащить из словаря.

In [ ]:
# Зарплата
df["salary"] =  df["salary"].map(lambda d : ast.literal_eval(d))
df_buffer = df.join(pd.DataFrame(df["salary"].to_dict()).T)
df_buffer = df_buffer.fillna(value=np.nan)  # Не у всех вакансий есть минимальная и максимальная зарплата

In [ ]:
df = df_buffer[['has_test', 'response_letter_required', 'accept_temporary', 'professional_roles', 'accept_incomplete_resumes', 'experience', 'employment', 'from', 'to', 'currency', 'gross']]
df.head()

Поскольку мы не можем заполнить отсутствующие значения from и to без нарушения логики датасета с помощью среднего или медианы (когда у вакансий минимальная может быть выше максимальной и т.п.), то заполним эти две колонки в одну следующим образом:
Если указана только минимальная, то устанавливаем её и как максимальную.
Если указана только максимальная, то устанавливаем её и как минимальную.

In [ ]:
df['to'] = df['to'].fillna(df['from'])
df['from'] = df['from'].fillna(df['to'])
df.head()

In [ ]:
# Профессия
df['professional_roles'] =  df['professional_roles'].map(lambda d : ast.literal_eval(d))

dict_buff = df['professional_roles'].to_dict()
for key in dict_buff:
    dict_buff[key] = dict(name = dict_buff[key][0]['name'])

df_buffer = df.join(pd.DataFrame(dict_buff).T)

In [ ]:
df = df_buffer[['name', 'from', 'to', 'currency', 'gross', 'has_test', 'response_letter_required', 'accept_temporary', 'accept_incomplete_resumes', 'experience', 'employment']]
df.head()

In [ ]:
# Опыт
df['experience'] =  df['experience'].map(lambda d : ast.literal_eval(d))

dict_buff = df['experience'].to_dict()
for key in dict_buff:
    dict_buff[key] = dict(exp_buff = dict_buff[key]['name'])

df_buffer = df.join(pd.DataFrame(dict_buff).T)

In [ ]:
df = df_buffer[['name', 'from', 'to', 'currency', 'gross', 'has_test', 'response_letter_required', 'accept_temporary', 'accept_incomplete_resumes', 'exp_buff', 'employment']]
df.rename(columns={'exp_buff': 'experience'}, inplace=True)
df.head()

In [ ]:
# Тип занятости
df['employment'] =  df['employment'].map(lambda d : ast.literal_eval(d))

dict_buff = df['employment'].to_dict()
for key in dict_buff:
    dict_buff[key] = dict(emp_buff = dict_buff[key]['name'])

df_buffer = df.join(pd.DataFrame(dict_buff).T)

In [ ]:
df = df_buffer[['name', 'from', 'to', 'currency', 'gross', 'has_test', 'response_letter_required', 'accept_temporary', 'accept_incomplete_resumes', 'experience', 'emp_buff']]
df.rename(columns={'emp_buff': 'employment'}, inplace=True)
df.head()

Последнее, что стоит сделать закодировать все True и False значения в числовой вариант 1 и 0, для простоты работы в будущем.

In [ ]:
df['gross'] = np.where(df['gross'] == False, 0, 1)
df['has_test'] = np.where(df['has_test'] == False, 0, 1)
df['response_letter_required'] = np.where(df['response_letter_required'] == False, 0, 1)
df['accept_temporary'] = np.where(df['accept_temporary'] == False, 0, 1)
df['accept_incomplete_resumes'] = np.where(df['accept_incomplete_resumes'] == False, 0, 1)
df.head()

На этом обработка спарсенного датасета окончена. Можно переходить к визуализации.

#### Визуализация

На этом этапе мы рассмотрим значение каждой колонки датасета, а так же посмотрим на интересные закономерности, которые можно в нём найти.

###### **Начнём с описания колонок датасета:**
**index** — уникальный id вакансии на hh.ru
**name** — профессия вакансии
**from** — минимальная зарплата
**to** — максимальная зарплата
**currency** — валюта выплаты
**gross** — указанна ли сумма до удержания налогов
**has_test** — нужно ли пройти тест перед отправкой резюме
**response_letter_required** — нужно ли сопроводительное письмо
**accept_temporary** — временная или постоянная работа
**accept_incomplete_resumes** — принимается ли неполное резюме
**experience** — требуемый опыт
**employment** — тип занятости

###### Общий осмотр данных по колонкам в датасете

Кажется интересным начать с того, чтобы взглянуть на соотношение валют в нашем датасете.

In [ ]:
currencies = df['currency'].value_counts()
currencies

Результат уже говорит о многом, однако всё же можно визуализировать этот момент.

In [ ]:
keys = ['RUB', 'USD']
data = [1986/2000, 14/2000]

palette_color = sb.color_palette('bright')
plt.pie(data, labels=keys, colors=palette_color, autopct='%.0f%%')
plt.show()

Раз у нас всего две валюты, то мы можем их закодировать, для простоты работы в будущем.
Пускай RUR будет равен 1, а USD 0.

In [ ]:
df['currency'] = np.where(df['currency'] == 'RUR', 1, 0)

Теперь лучше взглянуть на что-то, что может оказаться поинтереснее, на пример распределение по профессиям.

In [ ]:
professions = df['name'].value_counts()
professions

У нас вышло 130 профессий.
Можем визуализировать первые 10 из них, а все остальные загнать в "Другое"

In [ ]:
professions.head(10)

In [ ]:
keys = professions.head(10).keys()
data = [459/2000, 1105/2000, 91/2000, 91/2000, 66/2000, 52/2000, 50/2000, 44/2000, 42/2000, 36/2000]

palette_color = sb.color_palette('bright')
plt.pie(data, labels=keys, colors=palette_color, autopct='%.0f%%')
plt.show()

О чём из подобной графики можно судить?
В первую очередь о том, что самая востребованная профессия среди работодателей в финансовом секторе это менеджер по продажам. Говорить о причинах такой тенденции сложно, поскольку причины могут быть самыми различными, от большой текучки в сфере, до растущей потребности в более прямой продаже продукта на фоне растущей конкуренции.

Так же интересным элементом выделяется и разнообразие среди профессий. Более половины всех вакансий в выборке, это вакансии, необходимая для которых профессия встречается реже, чем в 36 случаях.

Теперь же, прежде чем окончательно перейти к выявлению закономерностей между данными, можем взглянуть на требование написания сопроводительного письма к резюме.

In [ ]:
letter = df['response_letter_required'].value_counts()
letter

Результат, так же как и в случае с валютами, вышел малоинтересным.
Но всё же стоит визуализировать.

In [ ]:
keys = ['Требуется письмо', 'Не требуется письмо']
data = [1964/2000, 36/2000]

palette_color = sb.color_palette('bright')
plt.pie(data, labels=keys, colors=palette_color, autopct='%.0f%%')
plt.show()

##### Взаимосвязь между данными